# APIM ❤️ OpenAI

## Backend pool Load Balancing lab
![flow](images/backend-pool-load-balancing.gif)

Playground to try the built-in load balancing [backend pool functionality of APIM](https://learn.microsoft.com/en-us/azure/api-management/backends?tabs=bicep) to either a list of Azure OpenAI endpoints or mock servers.

Notes:
- The backend pool uses round-robin by default
- But priority and weight based routing are also supported: Adjust the `priority` (the lower the number, the higher the priority) and `weight` parameters in the `openai_resources` variable
- The `retry` API Management policy initiates a retry to an available backend if an HTTP 429 status code is encountered

<a id='2'></a>
### 2️⃣ Create deployment using Terraform

This lab uses Terraform to declaratively define all the resources that will be deployed. Change the [variables.tf](variables.tf) directly to try different configurations.

In [ ]:
! terraform init
! terraform apply -auto-approve

<a id='sdk'></a>
### 🧪 Test the API using the Azure OpenAI Python SDK

Repeat the same test using the Python SDK to ensure compatibility.

In [74]:
apim_resource_gateway_url = ! terraform output -raw apim_resource_gateway_url
apim_resource_gateway_url = apim_resource_gateway_url.n
print("👉🏻 APIM Resource Gateway URL: ", apim_resource_gateway_url)

app_insights_app_id = ! terraform output -raw app_insights_app_id
app_insights_app_id = app_insights_app_id.n
print("👉🏻 Application Insights App ID: ", app_insights_app_id)

apim_subscription_key_1 = ! terraform output -raw apim_subscription_key_1
apim_subscription_key_1 = apim_subscription_key_1.n
print("👉🏻 APIM Subscription Key 1: ", apim_subscription_key_1)

apim_subscription_key_2 = ! terraform output -raw apim_subscription_key_2
apim_subscription_key_2 = apim_subscription_key_2.n
print("👉🏻 APIM Subscription Key 2: ", apim_subscription_key_2)

apim_subscription_key_3 = ! terraform output -raw apim_subscription_key_3
apim_subscription_key_3 = apim_subscription_key_3.n
print("👉🏻 APIM Subscription Key 3: ", apim_subscription_key_3)

resource_group_name = ! terraform output -raw resource_group_name
resource_group_name = resource_group_name.n
print("👉🏻 Resource Group Name: ", resource_group_name)

app_insights_resource_name = ! terraform output -raw app_insights_resource_name
app_insights_resource_name = app_insights_resource_name.n
print("👉🏻 Application Insights Resource Name: ", app_insights_resource_name)

openai_api_version = "2024-10-21"
openai_model_name = "gpt-4o"
openai_deployment_name = "gpt-4o"

👉🏻 APIM Resource Gateway URL:  https://apim-external-310-uat.azure-api.net
👉🏻 Application Insights App ID:  94ed0e76-b7d1-45e2-8192-201041d59f22
👉🏻 APIM Subscription Key 1:  da212d02472a46c5a482aa004f607ec9
👉🏻 APIM Subscription Key 2:  5f6a91c46d85423295043550dcc06ff0
👉🏻 APIM Subscription Key 3:  1b8956be8f1641fa80c24f01b8058cdb
👉🏻 Resource Group Name:  rg-apim-genai-310-uat
👉🏻 Application Insights Resource Name:  app-insights


## ⚠️ Important note about API Management Subscriptions

You will need to go to the API Management instance in the Azure portal and click each one of the three subscriptions and save them to activate them. There might be a bug preventing subscriptions to be validated.

<a id='requests'></a>
### 🧪 Test the API using a direct HTTP call
Requests is an elegant and simple HTTP library for Python that will be used here to make raw API requests and inspect the responses. 

You will not see HTTP 429s returned as API Management's `retry` policy will select an available backend. If no backends are viable, an HTTP 503 will be returned.

Tip: Use the [tracing tool](../../tools/tracing.ipynb) to track the behavior of the backend pool.

In [75]:
import time
import os
import json
import datetime
import requests

runs = 10
url = apim_resource_gateway_url + "/openai/deployments/" + openai_deployment_name + "/chat/completions?api-version=" + openai_api_version
api_runs = []

for i in range(runs):
    print("▶️ Run:", i+1, "/", runs)
    

    messages={"messages":[
        {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
        {"role": "user", "content": "Can you tell me the time, please?"}
    ]}

    start_time = time.time()
    response = requests.post(url, headers = {'api-key':apim_subscription_key_1}, json = messages)
    response_time = time.time() - start_time
    
    print(f"⌚ {response_time:.2f} seconds")
    # Check the response status code and apply formatting
    if 200 <= response.status_code < 300:
        status_code_str = '\x1b[1;32m' + str(response.status_code) + " - " + response.reason + '\x1b[0m'  # Bold and green
    elif response.status_code >= 400:
        status_code_str = '\x1b[1;31m' + str(response.status_code) + " - " + response.reason + '\x1b[0m'  # Bold and red
    else:
        status_code_str = str(response.status_code)  # No formatting

    # Print the response status with the appropriate formatting
    print("Response status:", status_code_str)
    
    print("Response headers:", response.headers)
    
    if "x-ms-region" in response.headers:
        print("x-ms-region:", '\x1b[1;31m'+response.headers.get("x-ms-region")+'\x1b[0m') # this header is useful to determine the region of the backend that served the request
        api_runs.append((response_time, response.headers.get("x-ms-region")))
    
    if (response.status_code == 200):
        data = json.loads(response.text)
        print("Token usage:", data.get("usage"), "\n")
        print("💬 ", data.get("choices")[0].get("message").get("content"), "\n")
    else:
        print(response.text)   

▶️ Run: 1 / 10
⌚ 3.50 seconds
Response status: 200 - OK
Response headers: {'Content-Type': 'application/json', 'Date': 'Wed, 18 Dec 2024 12:17:10 GMT', 'Cache-Control': 'private', 'Content-Encoding': 'gzip', 'Transfer-Encoding': 'chunked', 'Vary': 'Accept-Encoding', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'apim-request-id': 'eb52830e-e6bc-41dc-8480-657d3c4a6eec', 'x-ratelimit-remaining-requests': '79', 'x-accel-buffering': 'no', 'x-ms-rai-invoked': 'true', 'X-Request-ID': 'e0a46f8a-fee0-486b-b388-20e24824f2ec', 'X-Content-Type-Options': 'nosniff', 'azureml-model-session': 'd054-20241115143238', 'x-ms-region': 'UK South', 'x-envoy-upstream-service-time': '668', 'x-ms-client-request-id': 'Not-Set', 'x-ratelimit-remaining-tokens': '7340', 'Request-Context': 'appId=cid-v1:94ed0e76-b7d1-45e2-8192-201041d59f22'}
x-ms-region: UK South
Token usage: {'completion_tokens': 27, 'prompt_tokens': 30, 'total_tokens': 57} 

💬  Oh sure, let me just check my non-exis

<a id='sdk'></a>
### 🧪 Test the API using the Azure OpenAI Python SDK

Repeat the same test using the Python SDK to ensure compatibility.

In [76]:
import time
from openai import AzureOpenAI

runs = 9
sleep_time_ms = 0

for i in range(runs):
    print("▶️ Run: ", i+1)

    messages=[
        {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
        {"role": "user", "content": "Can you tell me the time, please?"}
    ]

    client = AzureOpenAI(
        azure_endpoint=apim_resource_gateway_url,
        api_key=apim_subscription_key_3,
        api_version=openai_api_version
    )

    start_time = time.time()

    response = client.chat.completions.create(model=openai_model_name, messages=messages)
    
    response_time = time.time() - start_time
    print(f"⌚ {response_time:.2f} seconds")
    print("💬 ", response.choices[0].message.content)
    time.sleep(sleep_time_ms/1000)


▶️ Run:  1
⌚ 3.26 seconds
💬  Oh sure, let me just lean over and check my invisible watch... Honestly, it's whatever time you want it to be. Whenever you ask an AI, the answer is 'time to buy a watch'!
▶️ Run:  2
⌚ 1.81 seconds
💬  Oh, absolutely! I'm looking at my magical, non-existent clock right now, and it says... drumroll, please... "somewhere between yesterday and tomorrow." What a revelation, right? If only I had access to actual time like your phone or watch. But alas, here we are!
▶️ Run:  3
⌚ 1.39 seconds
💬  Oh, sure, because I can totally access the current time and magically tell you, even though I'm just a text-based AI with no concept of time. Maybe try looking at a clock, your phone, or some other real-world device that actually knows what time it is!
▶️ Run:  4
⌚ 1.37 seconds
💬  Of course! Let me just check my imaginary wristwatch...oh wait, I'm just text on a screen. You'll have to look at a real clock for that information.
▶️ Run:  5
⌚ 34.83 seconds
💬  Oh, because I tot

<a id='kql'></a>
### 🔍 Analyze Application Insights requests

With this query you can get the request and response details including the prompt and the OpenAI completion. It also returns token counters.

In [61]:
import pandas as pd

query = "\"" + "requests  \
| project timestamp, duration, customDimensions \
| extend duration = round(duration, 2) \
| extend parsedCustomDimensions = parse_json(customDimensions) \
| extend apiName = tostring(parsedCustomDimensions.['API Name']) \
| extend apimSubscription = tostring(parsedCustomDimensions.['Subscription Name']) \
| extend userAgent = tostring(parsedCustomDimensions.['Request-User-agent']) \
| extend request_json = tostring(parsedCustomDimensions.['Request-Body']) \
| extend request = parse_json(request_json) \
| extend model = tostring(request.['model']) \
| extend messages = tostring(request.['messages']) \
| extend region = tostring(parsedCustomDimensions.['Response-x-ms-region']) \
| extend remainingTokens = tostring(parsedCustomDimensions.['Response-x-ratelimit-remaining-tokens']) \
| extend remainingRequests = tostring(parsedCustomDimensions.['Response-x-ratelimit-remaining-requests']) \
| extend response_json = tostring(parsedCustomDimensions.['Response-Body']) \
| extend response = parse_json(response_json) \
| extend promptTokens = tostring(response.['usage'].['prompt_tokens']) \
| extend completionTokens = tostring(response.['usage'].['completion_tokens']) \
| extend totalTokens = tostring(response.['usage'].['total_tokens']) \
| extend completion = tostring(response.['choices'][0].['message'].['content']) \
| project timestamp, apiName, apimSubscription, duration, userAgent, model, messages, completion, region, promptTokens, completionTokens, totalTokens, remainingTokens, remainingRequests \
| order by timestamp desc" + "\""

result_stdout = !  az monitor app-insights query --app {app_insights_app_id} --analytics-query {query} 
result = json.loads(result_stdout.n)

table = result.get('tables')[0]
pd.DataFrame(table.get("rows"), columns=[col.get("name") for col in table.get('columns')])


,timestamp,apiName,apimSubscription,duration,userAgent,model,messages,completion,region,promptTokens,completionTokens,totalTokens,remainingTokens,remainingRequests
0,2024-12-18T10:40:18.7930736Z,api-azure-openai,6d7e2ee1-4338-4be9-9053-7c58bb308677,1091.32,AzureOpenAI/Python 1.41.0,gpt-4o,"[{""role"":""system"",""content"":""You are a sarcast...","Oh sure, let me just reach into the cosmos and...",France Central,30,33,63,7340,79
1,2024-12-18T10:40:16.4014444Z,api-azure-openai,6d7e2ee1-4338-4be9-9053-7c58bb308677,1617.80,AzureOpenAI/Python 1.41.0,gpt-4o,"[{""role"":""system"",""content"":""You are a sarcast...","Oh sure, let me just gaze into my crystal ball...",UK South,30,42,72,2720,72
2,2024-12-18T10:40:14.5775188Z,api-azure-openai,6d7e2ee1-4338-4be9-9053-7c58bb308677,1052.37,AzureOpenAI/Python 1.41.0,gpt-4o,"[{""role"":""system"",""content"":""You are a sarcast...","Oh sure, let me just look at my nonexistent wa...",UK South,30,29,59,3380,73
3,2024-12-18T10:40:12.4827996Z,api-azure-openai,6d7e2ee1-4338-4be9-9053-7c58bb308677,1339.41,AzureOpenAI/Python 1.41.0,gpt-4o,"[{""role"":""system"",""content"":""You are a sarcast...","Sure, it's exactly the same time it was when y...",UK South,30,43,73,4040,74
4,2024-12-18T10:40:10.2092307Z,api-azure-openai,6d7e2ee1-4338-4be9-9053-7c58bb308677,1483.04,AzureOpenAI/Python 1.41.0,gpt-4o,"[{""role"":""system"",""content"":""You are a sarcast...","Oh sure, I definitely have a magical ability t...",UK South,30,54,84,4700,75
5,2024-12-18T10:40:07.9016084Z,api-azure-openai,6d7e2ee1-4338-4be9-9053-7c58bb308677,1549.19,AzureOpenAI/Python 1.41.0,gpt-4o,"[{""role"":""system"",""content"":""You are a sarcast...","Oh, of course! Because I'm totally equipped wi...",UK South,30,52,82,5360,76
6,2024-12-18T10:40:05.6159706Z,api-azure-openai,6d7e2ee1-4338-4be9-9053-7c58bb308677,1500.89,AzureOpenAI/Python 1.41.0,gpt-4o,"[{""role"":""system"",""content"":""You are a sarcast...","Oh, of course, I have all the time in the worl...",UK South,30,58,88,6020,77
7,2024-12-18T10:40:03.271906Z,api-azure-openai,6d7e2ee1-4338-4be9-9053-7c58bb308677,1580.71,AzureOpenAI/Python 1.41.0,gpt-4o,"[{""role"":""system"",""content"":""You are a sarcast...",Sure! Just as soon as I install a clock functi...,UK South,30,42,72,6680,78
8,2024-12-18T10:40:01.4217747Z,api-azure-openai,6d7e2ee1-4338-4be9-9053-7c58bb308677,1087.63,AzureOpenAI/Python 1.41.0,gpt-4o,"[{""role"":""system"",""content"":""You are a sarcast...","Oh sure, because I'm just a talking clock here...",UK South,30,33,63,7340,79
9,2024-12-18T10:35:49.8130934Z,api-azure-openai,bc972f9b-3fa6-4790-9454-72ebf6298d94,5581.61,python-requests/2.32.3,,"[{""role"":""system"",""content"":""You are a sarcast...","Oh, of course! Let me just check my highly adv...",Sweden Central,30,65,95,7340,79


<a id='portal'></a>
### 🔍 Open the workbook in the Azure Portal

Go to the application insights resource and under the Monitoring section select the Workbooks blade. You should see the OpenAI Usage Analysis workbook with the above query and some others to check token counts, performance, failures, etc.

<a id='sdk'></a>
### 🧪 Execute multiple runs for each subscription using the Azure OpenAI Python SDK

We will send requests for each subscription. Adjust the `sleep_time_ms` and the number of `runs` to your test scenario.


In [77]:
import time
from openai import AzureOpenAI
runs = 5

for i in range(runs):
    print("▶️ Run: ", i+1)
    messages=[
        {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
        {"role": "user", "content": "Can you tell me the time, please?"}
    ]
    client = AzureOpenAI(azure_endpoint=apim_resource_gateway_url, api_key=apim_subscription_key_1, api_version=openai_api_version)
    response = client.chat.completions.create(model=openai_model_name, messages=messages, extra_headers={"x-user-id": "alex"})
    print("💬 ","for subscription 1: ", response.choices[0].message.content)

    client = AzureOpenAI(azure_endpoint=apim_resource_gateway_url, api_key=apim_subscription_key_2, api_version=openai_api_version)
    response = client.chat.completions.create(model=openai_model_name, messages=messages, extra_headers={"x-user-id": "alex"})
    print("💬 ","for subscription 2: ", response.choices[0].message.content)

    client = AzureOpenAI(azure_endpoint=apim_resource_gateway_url, api_key=apim_subscription_key_3, api_version=openai_api_version)
    response = client.chat.completions.create(model=openai_model_name, messages=messages, extra_headers={"x-user-id": "alex"})
    print("💬 ","for subscription 3: ", response.choices[0].message.content)


▶️ Run:  1
💬  for subscription 1:  Oh, absolutely! Just let me calibrate my quantum clock... and... oh wait, I'm just a text-based assistant and can’t actually tell the time. But here's a groundbreaking idea: have you tried looking at a clock?
💬  for subscription 2:  Oh, absolutely, let me just consult my magical time-telling abilities. Oops! Looks like they're malfunctioning, because I actually can't keep track of real-time information. Maybe try looking at a clock or your phone? They're usually pretty reliable for that sort of thing!
💬  for subscription 3:  Oh sure, let me just check my non-existent watch. Unfortunately, I can't tell you the current time. Maybe try looking at literally any clock around you or the device you’re using to read this!
▶️ Run:  2
💬  for subscription 1:  Oh, absolutely! Let me just check my non-existent watch. It's currently... wait for it... "sometime past whenever it was earlier." Handy, right? In all seriousness, you might want to check a clock or a devi

<a id='kql'></a>
### 🔍 Analyze Application Insights custom metrics with a KQL query

With this query you can get the custom metrics that were emitted by Azure APIM.

In [64]:
import pandas as pd
import json

query = "\"" + "customMetrics  \
| where name == 'Total Tokens' \
| extend parsedCustomDimensions = parse_json(customDimensions) \
| extend clientIP = tostring(parsedCustomDimensions.['Client IP']) \
| extend apiId = tostring(parsedCustomDimensions.['API ID']) \
| extend apimSubscription = tostring(parsedCustomDimensions.['Subscription ID']) \
| extend UserId = tostring(parsedCustomDimensions.['User ID']) \
| project timestamp, value, clientIP, apiId, apimSubscription, UserId \
| order by timestamp asc" + "\""

result_stdout = ! az monitor app-insights query --app {app_insights_resource_name} -g {resource_group_name} --analytics-query {query} 
result = json.loads(result_stdout.n)

table = result.get('tables')[0]
df = pd.DataFrame(table.get("rows"), columns=[col.get("name") for col in table.get('columns')])
df['timestamp'] = pd.to_datetime(df['timestamp']).dt.strftime('%H:%M')

df


,timestamp,value,clientIP,apiId,apimSubscription,UserId
0,10:35,731,176.177.25.47,api-azure-openai,bc972f9b-3fa6-4790-9454-72ebf6298d94,N/A
1,10:40,656,176.177.25.47,api-azure-openai,6d7e2ee1-4338-4be9-9053-7c58bb308677,N/A
2,10:41,52,176.177.25.47,api-azure-openai,6d7e2ee1-4338-4be9-9053-7c58bb308677,alex
3,10:41,135,176.177.25.47,api-azure-openai,bc972f9b-3fa6-4790-9454-72ebf6298d94,alex
4,10:41,141,176.177.25.47,api-azure-openai,7cc7ca81-c3df-482e-b917-8c1b9da0b79e,alex
